## Song Metadata DB 

#### Found Here: https://song-metadata-default-rtdb.firebaseio.com/

Purpose: To store basic information about each track, useful for general queries and indexing.

Columns / Variables : track_id, artists, album_name, track_name, popularity, duration_ms, explicit, track_genre.

 

##  Audio Elements DB

#### Found Here: https://audio-elements-e5904-default-rtdb.firebaseio.com/

Purpose: To store detailed audio features of each song, useful for analysis and recommendation algorithms.

Columns/ Variables : track_id, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature.

In [ ]:
#requirements
pip install firebase-admin

In [2]:
#Import Libraries/Modules
import os
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db

In [3]:
#Load data
file_path = 'spotify_data.csv'
spotify_data = pd.read_csv(file_path)

#Split the data into two DF's
song_metadata_cols = ['track_id', 'artists', 'album_name', 'track_name', 'track_genre']
audio_elements_cols = ['track_id', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

#Assign new Df's
song_metadata = spotify_data[song_metadata_cols]
audio_elements = spotify_data[audio_elements_cols]


## Instructions to find or generate the JSON Firebase DB Key file:

1. Go to the Firebase Console: Visit Firebase Console and select your project.

2. Access Project Settings: Click on the gear icon next to "Project Overview" in the sidebar and select "Project settings."

3. Navigate to Service Accounts: In the project settings, go to the "Service accounts" tab.

4. Generate a New Private Key: Click on the "Generate new private key" button. A JSON file will be downloaded to your computer.

5. The path you need to provide in the script is the location where this JSON file is saved

## On macOS or Linux, use the export command in the terminal: 


### Option 1:
export SONG_METADATA_CREDENTIALS=path/to/song_metadata_credentials.json

export AUDIO_ELEMENTS_CREDENTIALS=path/to/audio_elements_credentials.json

### Option 2:

Set the os variabels directly in the environment using  'os.environ'.  Replace path\to\song_metadata_credentials.json and path\to\audio_elements_credentials.json with the actual paths to your own credentials files.

Each collaborator can set these environment variables on their own machine, ensuring that sensitive information is kept secure and out of version control.



In [4]:
os.environ['SONG_METADATA_CREDENTIALS'] = 'YOUR OWN PATH.JSON'

os.environ['AUDIO_ELEMENTS_CREDENTIALS'] = 'YOUR OWN PATH.JSON'

In [5]:
#Initialize Firebase Admin SDK for both databases using environment variables
cred_song_metadata = credentials.Certificate(os.environ.get('SONG_METADATA_CREDENTIALS'))
app_song_metadata = firebase_admin.initialize_app(cred_song_metadata, {
    'databaseURL': 'https://song-metadata-default-rtdb.firebaseio.com/'
}, name='song_metadata')

cred_audio_elements = credentials.Certificate(os.environ.get('AUDIO_ELEMENTS_CREDENTIALS'))
app_audio_elements = firebase_admin.initialize_app(cred_audio_elements, {
    'databaseURL': 'https://audio-elements-e5904-default-rtdb.firebaseio.com/'
}, name='audio_elements')


In [9]:
#Define a hash function
def hash_function(track_id):
    return hash(track_id) % 2 #hash function based on the last digit of track_id, even or odd


In [10]:
#Modify the upload function
def upload_data_to_firebase(df, db_base_url, db_name):
    for index, row in df.iterrows():
        row_dict = row.to_dict()
        for key, value in row_dict.items():
            if pd.isna(value) or value in [float('inf'), float('-inf')]:
                row_dict[key] = None

        track_id = str(row['track_id'])
        hash_value = hash_function(track_id)
        sub_table = f'{db_name}_{hash_value}'
        ref = db.reference(f'/{sub_table}', app=firebase_admin.get_app(db_name))

        if ref.child(track_id).get() is None:
            ref.child(track_id).set(row_dict)
        else:
            print(f"Skipping existing record: {track_id}")


In [11]:
#Upload song metadata to Firebase
upload_data_to_firebase(song_metadata, 'https://song-metadata-default-rtdb.firebaseio.com/', 'song_metadata')

Skipping existing record: 0CDucx9lKxuCZplLXUz0iX
Skipping existing record: 2aibwv5hGXSgw7Yru8IYTO
Skipping existing record: 5E30LdtzQTGqRvNd7l6kG5
Skipping existing record: 2K7xn816oNHJZ0aVqdQsha
Skipping existing record: 2QjOHCTQ1Jl3zawyYOpxh6
Skipping existing record: 6rrKbzJGGDlSZgLphopS49
Skipping existing record: 0AUyNF6iFxMNQsNx2nhtrw
Skipping existing record: 75fT19n6ZQHr67Mkh86qfD
Skipping existing record: 1tRpraJDpEUl0HjnkMosqx
Skipping existing record: 2AEdT2ZKd14kzOouzdNOUj
Skipping existing record: 23PvWFdi76vER4p1e2Xroj
Skipping existing record: 5xUhs6dlfkcxZM8zq7riYF
Skipping existing record: 62pennq1pcVBFVW1b1CrFu
Skipping existing record: 1yZWmKmLfTrFCxHvhFsdoN
Skipping existing record: 0v8tmDMQrTMavluubqAJo4
Skipping existing record: 2UCoAOZGk6a4BHUBzlvfAO
Skipping existing record: 5SVYqPYpje5JDG2O969oz6
Skipping existing record: 12cG3BUpk6PzLAzWK0mryO
Skipping existing record: 34ldTowcv56E2DqtSaYBjP
Skipping existing record: 6QvYmrqFKRSVrcVgvoA7Ar
Skipping existing re

Skipping existing record: 2iwsJcPB5HIVTuHHwdTNO4
Skipping existing record: 6CkLcIjIAWHJp3nZWhNQvF
Skipping existing record: 4dI3UmOl7scnyCU4KgaaNO
Skipping existing record: 5WTKiRTdoDk07YJOP7y8DU
Skipping existing record: 3U6726RPDdw6JshdT9fGFM
Skipping existing record: 3KKmCToTNy7Mnvkd9k0vVR
Skipping existing record: 7n4Y9BCKLHi4hqI2NctuAq
Skipping existing record: 0ND3OzMoWDW8ciiobRGXxf
Skipping existing record: 5UFr6YU5nIp1ra63Yusbp7
Skipping existing record: 4sO7A5yF6SfqiHeINS61DE
Skipping existing record: 42Jkqzhm5Y7ab7f1d22GTq
Skipping existing record: 12AJRjW79qbZktCDpMifs5
Skipping existing record: 7D9CMVj8TeaozjxYCIE5oF
Skipping existing record: 5cfct39EqJAkG0cqRaARz1
Skipping existing record: 1lYHv3SEkE6wKfZRe0HMOG
Skipping existing record: 04bNiW1CiMHWM0kSDvbLzp
Skipping existing record: 2mLYjGJ1Fl4qNept7opf7Y
Skipping existing record: 6lVtX3nLHyWZP7aqU9HuVW
Skipping existing record: 6JN0cjFT0zDRYxzqcKrvwf
Skipping existing record: 0QBrw0zGpCa8gKIJd2KDGo
Skipping existing re

Skipping existing record: 5UWwZ5lm5PKu6eKsHAGxOk
Skipping existing record: 57iDDD9N9tTWe75x6qhStw
Skipping existing record: 5Gs6NY7QTgEq77V3DtPagv
Skipping existing record: 3CMA6bdFwobZpvStfR2jls
Skipping existing record: 4JuKzBOcFT6ZDX1bINNI1C
Skipping existing record: 7em6T7mETpDEzyPg2UArtk
Skipping existing record: 54RIrgjhCmUvAwmUuDqJTA
Skipping existing record: 7JOIlJ9KSPIr8SRK4zl8zj
Skipping existing record: 5Zk9paAVhGvRR1N0s3e0BA
Skipping existing record: 5zLtABAGH8rxBXOyEgqCgl
Skipping existing record: 0xD23i0X1WjqMSYIxRfO80
Skipping existing record: 7lGKEWMXVWWTt3X71Bv44I
Skipping existing record: 3eHYnUFpEZjnTQag1EVxHC
Skipping existing record: 0815caqt2Lytro5EIzMufT
Skipping existing record: 35lxFAny9XDBkvE6r9j4uG
Skipping existing record: 6G8xvLFNkXy3HM5EbQI6c8
Skipping existing record: 12g9IeQzX7xECLNxz8dpb5
Skipping existing record: 5M5TD1Tlpcgq0Fj1l6Mz6Z
Skipping existing record: 46gSk82duJtX3TTA182ruG
Skipping existing record: 0ADZ5dmXhlfzjMw6lefoPl
Skipping existing re

Skipping existing record: 2oxkdzlJjiFWj8lfsZkaCg
Skipping existing record: 0Sjz8pTu8vcVvGP5rtyh3Q
Skipping existing record: 07VBqWH6NG8L2ad3l84UKk
Skipping existing record: 40iocOzytrqaRn5RbGPDNp
Skipping existing record: 113xf7t4qNM7038YJvauik
Skipping existing record: 7saeg1XSIx0RhNMHY5km2e
Skipping existing record: 1jq28NGw6wdtFKx8MBPy6C
Skipping existing record: 5e3YOg6fIkP0wD5TyxcHOH
Skipping existing record: 2MLHyLy5z5l5YRp7momlgw
Skipping existing record: 5pKCDm2fw4k6D6C5Rk646C
Skipping existing record: 3UULkHdmLDqBDPmBYltoRE
Skipping existing record: 6A6PDawvZIfnJHW40sBHzz
Skipping existing record: 73CKjW3vsUXRpy3NnX4H7F
Skipping existing record: 5so3Q5ihUoWp79tmIm8z7l
Skipping existing record: 6TfBA04WJ3X1d1wXhaCFVT
Skipping existing record: 4WmMZtbFw54OYKrqVKVupx
Skipping existing record: 0wMGxaOeu5zZfbx3gWctMx
Skipping existing record: 6XEbIRw3DVFb6vQftaewOD
Skipping existing record: 6VQW8Ye28Crwz90BIexsrF
Skipping existing record: 03DgLHhA7HCUB8aBTNqOwX
Skipping existing re

Skipping existing record: 5wSagCWflNwJ1xM59ZKw6O
Skipping existing record: 1QJWQRagviM87ff3wQjBkP
Skipping existing record: 2HegWuw33lBqopjopozt7N
Skipping existing record: 1aKSNq3Skm9MIIksjx5bFy
Skipping existing record: 4k2AjlWruIygEiZyJEy01I
Skipping existing record: 6zciWZePv9ohzMshMJn7uu
Skipping existing record: 6JMRXqVpDD1Xu2lAoFOVbS
Skipping existing record: 3QI2FbgxMG42vJjqNEHOP3
Skipping existing record: 2gOhhHSjm3iw4NFY43carm
Skipping existing record: 2NV0oNAYgjlN27SeNIo8bL
Skipping existing record: 2ZTuf3VIwB1EKuzwzFdJTT
Skipping existing record: 4dmgrfGknhKKaozk5U3JXL
Skipping existing record: 0OwkWUJHTcxJSxlprEGd9w
Skipping existing record: 098WwIoAtjDQwcpuEHAlrr
Skipping existing record: 6ZkeNE5zlcl13PbnzXdw38
Skipping existing record: 2hRNxkgCScd5yxbTsrGQR4
Skipping existing record: 4to8RSl1qZa7q3X8z4nOk3
Skipping existing record: 2m2QV4YsDoVNQ9Y6THdSg4
Skipping existing record: 7gVNP7rI9UBZndge0ulKfL
Skipping existing record: 2ScU6iEvgb0TIuKiyem9rg
Skipping existing re

...

Skipping existing record: 4EnSchPRIh9xWOb2SIrRQU
Skipping existing record: 0cKx33I7DYZkEPcZVIgrEB
Skipping existing record: 7ix05uGBD7ryqUtGgd5sD0
Skipping existing record: 79UzFFBn4dFovvqh9bKanm
Skipping existing record: 1voNki3zm1VcvtLeAJtkGK
Skipping existing record: 7hjbOIcQnULsxDoS2CPmdQ
Skipping existing record: 7Kwki4PiBWp3b4Re5Rcdes
Skipping existing record: 1vkpmgiYKoej9sA5y70KZd
Skipping existing record: 0Wf54ocGy08lcTan5klPqX
Skipping existing record: 071BITLZcUvNiEspOR1BgI
Skipping existing record: 22gy6K7sdAOqqYFMItp1Wl
Skipping existing record: 1VIzgYvo4Got947F1EMGIo
Skipping existing record: 2VnlJCQMphFJUyYR5p7da2
Skipping existing record: 0RU3sUD0wsMTKSV4gvppOS
Skipping existing record: 239SrBPZi0xs1OQZmedr6q
Skipping existing record: 3bjfZcKL5HZOahNAA6v5zw
Skipping existing record: 0iZq53rxJMgd4SeGFH86Zo
Skipping existing record: 4dw78431bSpfPg4SfVWQNr
Skipping existing record: 7H0MOWplwc6gf2AD1vTSYi
Skipping existing record: 7Ds7GEIdCocSvO8NqHieuB
Skipping existing re

Skipping existing record: 6vM3LS50ZghrtkTKBvm2Kd
Skipping existing record: 6T0e1wKkONiIZQ1FTjMlGZ
Skipping existing record: 2PVA13ykYYHeXKlIsx2Vjk
Skipping existing record: 4e6fdJaczDlNjMvNwVMSVD
Skipping existing record: 5ZQUoZUqQ1D7QiSOmtQCoI
Skipping existing record: 68wA3be5nyIqyHvukORvqK
Skipping existing record: 5dSc5BbU8wbGY0dHVpf4Ys
Skipping existing record: 6N2dua8p7nL9T67WITwaIY
Skipping existing record: 3GmVvoNGooqTx3fV2H1rjU
Skipping existing record: 0aeqKlSGQgi6Dwr87N3LNq
Skipping existing record: 3CvfYeaJX8JAT1tF5IzVlG
Skipping existing record: 1rBJb3j8zvMlwIxmozOE7d
Skipping existing record: 4LTFOJjQt0wSunlfvIuCYe
Skipping existing record: 512rk6iIuzhzOtBGHTUVUS
Skipping existing record: 2o7xb3qaBMnGicq3T4XEWv
Skipping existing record: 2ZFx4terxlW3FNY25kpy71
Skipping existing record: 5DUOMqtIps27xFZvCxGWSD
Skipping existing record: 5x4wjHroFa5k0gUCzpYZAn
Skipping existing record: 3LNJXM1TJlksK5WnabNqDg
Skipping existing record: 2a1dpnaLw3CmTu9ofgwLE3
Skipping existing re

Skipping existing record: 2HB0RLSCaKALpRHFw7PcOy
Skipping existing record: 0wYVNC09iLBVIqHB07Hckt
Skipping existing record: 0P2sfcfRD0lWw4b8eXegmR
Skipping existing record: 5MuWGLrqeqYEcpqWMq5M1w
Skipping existing record: 0zjORbo98bRxMmyS3wzeKW
Skipping existing record: 4cfEA7zQhaQpkhUFonoOPw
Skipping existing record: 5p144lbzJw9kMCMOG61SF3
Skipping existing record: 0bMzCNqq4U5pQakqcyfJHR
Skipping existing record: 1oOOiIOR4OXntHkpSvytwb
Skipping existing record: 2CfNnviw8KsMGGcJ5RtZ61
Skipping existing record: 0CqCpm4LLQbHUX5ysizWJS
Skipping existing record: 0XHnYbXteyBDfVvk9EGaPu
Skipping existing record: 3n3turoo9rMhlrjh90lkyI
Skipping existing record: 7Dv1rYPkx3v2QVCPUY0yhj
Skipping existing record: 4s5YiAfgPWyq37WukmceRI
Skipping existing record: 2zI4py4xY6v4vIrAlR6GtP
Skipping existing record: 4PTIAmRG2ckFNySN9FArFv
Skipping existing record: 2sZ0uw5ZgibFVCRWmpWBRK
Skipping existing record: 79cxnmnGiC0qZfxi5ogp4j
Skipping existing record: 17C4oPQ3sTpF11pi9yvGcs
Skipping existing re

Skipping existing record: 73fRMyckZKtemX14eku8uu
Skipping existing record: 294JL3Xexu3sVRH9tpn60G
Skipping existing record: 1PrXSvX8N0SlQNQ4jMrCSb
Skipping existing record: 79OUttYM7WQUzEtsugTwEX
Skipping existing record: 5i9YJPhJo1RrWHKjkbkgmk
Skipping existing record: 27LtwN20Rvhi6LVmUaaucZ
Skipping existing record: 006tmNZLXEXPqdb23wwSN1
Skipping existing record: 4gBNgq0v7fG5kGC857Pb8M
Skipping existing record: 2LxGrlboLTVvmegFyRtdP5
Skipping existing record: 6NNiVBkstx4zWrQVbEpZ8I
Skipping existing record: 4hv9mEWi4911k1uhU4fPEH
Skipping existing record: 7J2JnVWhxySGf48iJgk6pv
Skipping existing record: 1oFnf7O5pRtgTt1U6cBxHl
Skipping existing record: 2u1B87jh7hcE0ImLMfHNji
Skipping existing record: 0jdfbvSdaWvxfAlD20TtNc
Skipping existing record: 36hTKOgo5Ra03y0GlpJPd3
Skipping existing record: 56AD3PBYdPepniv4VtsV7s
Skipping existing record: 6E9deAT9PHY3xwZwD0fvhZ
Skipping existing record: 3lIZy99mnEArB9FWVoxH3C
Skipping existing record: 2ol1ST9LqK6gR7QOp03exs
Skipping existing re

Skipping existing record: 5oiNJJdKqJ5Vfwion1dR1n
Skipping existing record: 35sVDinWrh5nYjpzPXe4qz
Skipping existing record: 4wiv0itDHcgSB8IjOxn4SA
Skipping existing record: 6p9Frj8pPOCWol6HMFihSs
Skipping existing record: 1d6BkMki9M6K0yp9OLBzOM
Skipping existing record: 2ZUEopklc3lUm4OXUPB6WS
Skipping existing record: 18Kt9lI9qQK5hUHtaIPOP2
Skipping existing record: 7FINWnbVBnZdja3kUIvptF
Skipping existing record: 2kDe0QRaCBKIh8QY64GDvK
Skipping existing record: 32DCSsnAxX8jpmu9w6C8DW
Skipping existing record: 3Zm6QMBYCwNZFUOmVBoVhA
Skipping existing record: 1VNP6bgTcqE6cxkU8Zpy21
Skipping existing record: 56ECskEyc7XqOCyRPruZ37
Skipping existing record: 5UivJ02lC3RH9MmAysIltU
Skipping existing record: 4jx6YFhGbw3Bkjn7o2KgzW
Skipping existing record: 19uV6FQniZrbWtDPvBisIB
Skipping existing record: 1saXUvvFlAQaefZUFVmhCn
Skipping existing record: 1Q5jFp1g2Ns4gBsHRpcqhu
Skipping existing record: 71dLJx3qHOTQMTvvoE2dmd
Skipping existing record: 6OG5TBCmuTOuWCzSGsETrE
Skipping existing re

In [12]:
#Upload audio elements to Firebase
upload_data_to_firebase(audio_elements, 'https://audio-elements-e5904-default-rtdb.firebaseio.com/', 'audio_elements')

Skipping existing record: 0CDucx9lKxuCZplLXUz0iX
Skipping existing record: 2aibwv5hGXSgw7Yru8IYTO
Skipping existing record: 5E30LdtzQTGqRvNd7l6kG5
Skipping existing record: 2K7xn816oNHJZ0aVqdQsha
Skipping existing record: 2QjOHCTQ1Jl3zawyYOpxh6
Skipping existing record: 6rrKbzJGGDlSZgLphopS49
Skipping existing record: 0AUyNF6iFxMNQsNx2nhtrw
Skipping existing record: 75fT19n6ZQHr67Mkh86qfD
Skipping existing record: 1tRpraJDpEUl0HjnkMosqx
Skipping existing record: 2AEdT2ZKd14kzOouzdNOUj
Skipping existing record: 23PvWFdi76vER4p1e2Xroj
Skipping existing record: 5xUhs6dlfkcxZM8zq7riYF
Skipping existing record: 62pennq1pcVBFVW1b1CrFu
Skipping existing record: 1yZWmKmLfTrFCxHvhFsdoN
Skipping existing record: 0v8tmDMQrTMavluubqAJo4
Skipping existing record: 2UCoAOZGk6a4BHUBzlvfAO
Skipping existing record: 5SVYqPYpje5JDG2O969oz6
Skipping existing record: 12cG3BUpk6PzLAzWK0mryO
Skipping existing record: 34ldTowcv56E2DqtSaYBjP
Skipping existing record: 6QvYmrqFKRSVrcVgvoA7Ar
Skipping existing re

Skipping existing record: 2iwsJcPB5HIVTuHHwdTNO4
Skipping existing record: 6CkLcIjIAWHJp3nZWhNQvF
Skipping existing record: 4dI3UmOl7scnyCU4KgaaNO
Skipping existing record: 5WTKiRTdoDk07YJOP7y8DU
Skipping existing record: 3U6726RPDdw6JshdT9fGFM
Skipping existing record: 3KKmCToTNy7Mnvkd9k0vVR
Skipping existing record: 7n4Y9BCKLHi4hqI2NctuAq
Skipping existing record: 0ND3OzMoWDW8ciiobRGXxf
Skipping existing record: 5UFr6YU5nIp1ra63Yusbp7
Skipping existing record: 4sO7A5yF6SfqiHeINS61DE
Skipping existing record: 42Jkqzhm5Y7ab7f1d22GTq
Skipping existing record: 12AJRjW79qbZktCDpMifs5
Skipping existing record: 7D9CMVj8TeaozjxYCIE5oF
Skipping existing record: 5cfct39EqJAkG0cqRaARz1
Skipping existing record: 1lYHv3SEkE6wKfZRe0HMOG
Skipping existing record: 04bNiW1CiMHWM0kSDvbLzp
Skipping existing record: 2mLYjGJ1Fl4qNept7opf7Y
Skipping existing record: 6lVtX3nLHyWZP7aqU9HuVW
Skipping existing record: 6JN0cjFT0zDRYxzqcKrvwf
Skipping existing record: 0QBrw0zGpCa8gKIJd2KDGo
Skipping existing re

Skipping existing record: 5UWwZ5lm5PKu6eKsHAGxOk
Skipping existing record: 57iDDD9N9tTWe75x6qhStw
Skipping existing record: 5Gs6NY7QTgEq77V3DtPagv
Skipping existing record: 3CMA6bdFwobZpvStfR2jls
Skipping existing record: 4JuKzBOcFT6ZDX1bINNI1C
Skipping existing record: 7em6T7mETpDEzyPg2UArtk
Skipping existing record: 54RIrgjhCmUvAwmUuDqJTA
Skipping existing record: 7JOIlJ9KSPIr8SRK4zl8zj
Skipping existing record: 5Zk9paAVhGvRR1N0s3e0BA
Skipping existing record: 5zLtABAGH8rxBXOyEgqCgl
Skipping existing record: 0xD23i0X1WjqMSYIxRfO80
Skipping existing record: 7lGKEWMXVWWTt3X71Bv44I
Skipping existing record: 3eHYnUFpEZjnTQag1EVxHC
Skipping existing record: 0815caqt2Lytro5EIzMufT
Skipping existing record: 35lxFAny9XDBkvE6r9j4uG
Skipping existing record: 6G8xvLFNkXy3HM5EbQI6c8
Skipping existing record: 12g9IeQzX7xECLNxz8dpb5
Skipping existing record: 5M5TD1Tlpcgq0Fj1l6Mz6Z
Skipping existing record: 46gSk82duJtX3TTA182ruG
Skipping existing record: 0ADZ5dmXhlfzjMw6lefoPl
Skipping existing re

Skipping existing record: 1HxNA8JjIIfVGqMr7Jm59f
Skipping existing record: 2oxkdzlJjiFWj8lfsZkaCg
Skipping existing record: 0Sjz8pTu8vcVvGP5rtyh3Q
Skipping existing record: 07VBqWH6NG8L2ad3l84UKk
Skipping existing record: 40iocOzytrqaRn5RbGPDNp
Skipping existing record: 113xf7t4qNM7038YJvauik
Skipping existing record: 7saeg1XSIx0RhNMHY5km2e
Skipping existing record: 1jq28NGw6wdtFKx8MBPy6C
Skipping existing record: 5e3YOg6fIkP0wD5TyxcHOH
Skipping existing record: 2MLHyLy5z5l5YRp7momlgw
Skipping existing record: 5pKCDm2fw4k6D6C5Rk646C
Skipping existing record: 3UULkHdmLDqBDPmBYltoRE
Skipping existing record: 6A6PDawvZIfnJHW40sBHzz
Skipping existing record: 73CKjW3vsUXRpy3NnX4H7F
Skipping existing record: 5so3Q5ihUoWp79tmIm8z7l
Skipping existing record: 6TfBA04WJ3X1d1wXhaCFVT
Skipping existing record: 4WmMZtbFw54OYKrqVKVupx
Skipping existing record: 0wMGxaOeu5zZfbx3gWctMx
Skipping existing record: 6XEbIRw3DVFb6vQftaewOD
Skipping existing record: 6VQW8Ye28Crwz90BIexsrF
Skipping existing re

Skipping existing record: 3JRi5bymnIQ2fhIetoOm3r
Skipping existing record: 5wSagCWflNwJ1xM59ZKw6O
Skipping existing record: 1QJWQRagviM87ff3wQjBkP
Skipping existing record: 2HegWuw33lBqopjopozt7N
Skipping existing record: 1aKSNq3Skm9MIIksjx5bFy
Skipping existing record: 4k2AjlWruIygEiZyJEy01I
Skipping existing record: 6zciWZePv9ohzMshMJn7uu
Skipping existing record: 6JMRXqVpDD1Xu2lAoFOVbS
Skipping existing record: 3QI2FbgxMG42vJjqNEHOP3
Skipping existing record: 2gOhhHSjm3iw4NFY43carm
Skipping existing record: 2NV0oNAYgjlN27SeNIo8bL
Skipping existing record: 2ZTuf3VIwB1EKuzwzFdJTT
Skipping existing record: 4dmgrfGknhKKaozk5U3JXL
Skipping existing record: 0OwkWUJHTcxJSxlprEGd9w
Skipping existing record: 098WwIoAtjDQwcpuEHAlrr
Skipping existing record: 6ZkeNE5zlcl13PbnzXdw38
Skipping existing record: 2hRNxkgCScd5yxbTsrGQR4
Skipping existing record: 4to8RSl1qZa7q3X8z4nOk3
Skipping existing record: 2m2QV4YsDoVNQ9Y6THdSg4
Skipping existing record: 7gVNP7rI9UBZndge0ulKfL
Skipping existing re

...

Skipping existing record: 6sH7LTzonbkNleLPvk1Yie
Skipping existing record: 4l1k3bK6OGwfRqbW5HiyT3
Skipping existing record: 54CEXt5kP0lFQTaRXYHXYw
Skipping existing record: 53XCa30Ua1FGl4VeAyCENK
Skipping existing record: 4EnSchPRIh9xWOb2SIrRQU
Skipping existing record: 0cKx33I7DYZkEPcZVIgrEB
Skipping existing record: 7ix05uGBD7ryqUtGgd5sD0
Skipping existing record: 79UzFFBn4dFovvqh9bKanm
Skipping existing record: 1voNki3zm1VcvtLeAJtkGK
Skipping existing record: 7hjbOIcQnULsxDoS2CPmdQ
Skipping existing record: 7Kwki4PiBWp3b4Re5Rcdes
Skipping existing record: 1vkpmgiYKoej9sA5y70KZd
Skipping existing record: 0Wf54ocGy08lcTan5klPqX
Skipping existing record: 071BITLZcUvNiEspOR1BgI
Skipping existing record: 22gy6K7sdAOqqYFMItp1Wl
Skipping existing record: 1VIzgYvo4Got947F1EMGIo
Skipping existing record: 2VnlJCQMphFJUyYR5p7da2
Skipping existing record: 0RU3sUD0wsMTKSV4gvppOS
Skipping existing record: 239SrBPZi0xs1OQZmedr6q
Skipping existing record: 3bjfZcKL5HZOahNAA6v5zw
Skipping existing re

Skipping existing record: 2fAMFVfSiadA8FUlQTx1bW
Skipping existing record: 2i6vQrYVULLekBceqmCNwT
Skipping existing record: 5fMy36xPHP4PEgsirxLmma
Skipping existing record: 1UktZnwuS1reRmRtli8QS9
Skipping existing record: 26rzkGV7pXYOIr4hUg8Mbb
Skipping existing record: 6vM3LS50ZghrtkTKBvm2Kd
Skipping existing record: 6T0e1wKkONiIZQ1FTjMlGZ
Skipping existing record: 2PVA13ykYYHeXKlIsx2Vjk
Skipping existing record: 4e6fdJaczDlNjMvNwVMSVD
Skipping existing record: 5ZQUoZUqQ1D7QiSOmtQCoI
Skipping existing record: 68wA3be5nyIqyHvukORvqK
Skipping existing record: 5dSc5BbU8wbGY0dHVpf4Ys
Skipping existing record: 6N2dua8p7nL9T67WITwaIY
Skipping existing record: 3GmVvoNGooqTx3fV2H1rjU
Skipping existing record: 0aeqKlSGQgi6Dwr87N3LNq
Skipping existing record: 3CvfYeaJX8JAT1tF5IzVlG
Skipping existing record: 1rBJb3j8zvMlwIxmozOE7d
Skipping existing record: 4LTFOJjQt0wSunlfvIuCYe
Skipping existing record: 512rk6iIuzhzOtBGHTUVUS
Skipping existing record: 2o7xb3qaBMnGicq3T4XEWv
Skipping existing re

Skipping existing record: 7KC9bnTvAbU3gJTBej0qP3
Skipping existing record: 7IiqF2tYiixnpBcFjkocjA
Skipping existing record: 4oaOiFsrOQFqRnIU3hg3cM
Skipping existing record: 12tw0TYIJMf1bowLBYWqTp
Skipping existing record: 7yfnNGWg10OgBjpGx1G2M7
Skipping existing record: 2s4AbiaTKGrKldnICBxaWW
Skipping existing record: 2HB0RLSCaKALpRHFw7PcOy
Skipping existing record: 0wYVNC09iLBVIqHB07Hckt
Skipping existing record: 0P2sfcfRD0lWw4b8eXegmR
Skipping existing record: 5MuWGLrqeqYEcpqWMq5M1w
Skipping existing record: 0zjORbo98bRxMmyS3wzeKW
Skipping existing record: 4cfEA7zQhaQpkhUFonoOPw
Skipping existing record: 5p144lbzJw9kMCMOG61SF3
Skipping existing record: 0bMzCNqq4U5pQakqcyfJHR
Skipping existing record: 1oOOiIOR4OXntHkpSvytwb
Skipping existing record: 2CfNnviw8KsMGGcJ5RtZ61
Skipping existing record: 0CqCpm4LLQbHUX5ysizWJS
Skipping existing record: 0XHnYbXteyBDfVvk9EGaPu
Skipping existing record: 3n3turoo9rMhlrjh90lkyI
Skipping existing record: 7Dv1rYPkx3v2QVCPUY0yhj
Skipping existing re

Skipping existing record: 4bQPTF342jVUgisDgJGMDo
Skipping existing record: 13DqxJiOQYYbgcFVhd8Rtk
Skipping existing record: 6hTlfgh8zGrpJ1ijRXxUGK
Skipping existing record: 5ptQepn6HP5jMllqHCmr4F
Skipping existing record: 6kHjMdhMOz1Y84X3DHvbni
Skipping existing record: 73fRMyckZKtemX14eku8uu
Skipping existing record: 294JL3Xexu3sVRH9tpn60G
Skipping existing record: 1PrXSvX8N0SlQNQ4jMrCSb
Skipping existing record: 79OUttYM7WQUzEtsugTwEX
Skipping existing record: 5i9YJPhJo1RrWHKjkbkgmk
Skipping existing record: 27LtwN20Rvhi6LVmUaaucZ
Skipping existing record: 006tmNZLXEXPqdb23wwSN1
Skipping existing record: 4gBNgq0v7fG5kGC857Pb8M
Skipping existing record: 2LxGrlboLTVvmegFyRtdP5
Skipping existing record: 6NNiVBkstx4zWrQVbEpZ8I
Skipping existing record: 4hv9mEWi4911k1uhU4fPEH
Skipping existing record: 7J2JnVWhxySGf48iJgk6pv
Skipping existing record: 1oFnf7O5pRtgTt1U6cBxHl
Skipping existing record: 2u1B87jh7hcE0ImLMfHNji
Skipping existing record: 0jdfbvSdaWvxfAlD20TtNc
Skipping existing re

Skipping existing record: 5WaioelSGekDk3UNQy8zaw
Skipping existing record: 3ExKJt4K8duUfEG49QWzPi
Skipping existing record: 14I9CpiA1CMTdLga8SKAjJ
Skipping existing record: 1ZJ8pODFM2y5DeSthUTBrC
Skipping existing record: 7FoR7NSegGv5kTSMhNC2sn
Skipping existing record: 5oiNJJdKqJ5Vfwion1dR1n
Skipping existing record: 35sVDinWrh5nYjpzPXe4qz
Skipping existing record: 4wiv0itDHcgSB8IjOxn4SA
Skipping existing record: 6p9Frj8pPOCWol6HMFihSs
Skipping existing record: 1d6BkMki9M6K0yp9OLBzOM
Skipping existing record: 2ZUEopklc3lUm4OXUPB6WS
Skipping existing record: 18Kt9lI9qQK5hUHtaIPOP2
Skipping existing record: 7FINWnbVBnZdja3kUIvptF
Skipping existing record: 2kDe0QRaCBKIh8QY64GDvK
Skipping existing record: 32DCSsnAxX8jpmu9w6C8DW
Skipping existing record: 3Zm6QMBYCwNZFUOmVBoVhA
Skipping existing record: 1VNP6bgTcqE6cxkU8Zpy21
Skipping existing record: 56ECskEyc7XqOCyRPruZ37
Skipping existing record: 5UivJ02lC3RH9MmAysIltU
Skipping existing record: 4jx6YFhGbw3Bkjn7o2KgzW
Skipping existing re

# THE END!